In [18]:
import datasets
from transformers import AutoTokenizer
from babilong_utils import TaskDataset, SentenceSampler, NoiseInjectionDataset

In [9]:
# !unzip ../data/tasks_1-20_v1-2.zip -d ../data/

In [12]:
train_path = "../data/tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_train.txt"
test_path = "../data/tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_test.txt"

noise_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1")

### Load task datasets

In [15]:
# task 
task_dataset_train = TaskDataset(train_path)
task_dataset_test = TaskDataset(test_path)

In [20]:
# background text
tokenizer = AutoTokenizer.from_pretrained('gpt2')

noise_sampler_train = SentenceSampler(noise_dataset['train'], tokenizer=tokenizer)
noise_sampler_test = SentenceSampler(noise_dataset['test'], tokenizer=tokenizer)

In [22]:
sample_size = 512               # max number of tokens in sample
dataset_train = NoiseInjectionDataset(task_dataset=task_dataset_train,
                                        noise_sampler=noise_sampler_train,
                                        tokenizer=tokenizer,
                                        sample_size=sample_size)

dataset_test = NoiseInjectionDataset(task_dataset=task_dataset_test,
                                        noise_sampler=noise_sampler_test,
                                        tokenizer=tokenizer,
                                        sample_size=sample_size)

In [26]:
sample = dataset_train[0]
sample.keys()

dict_keys(['facts', 'question', 'answer', 'references', 'background_text', 'input_tokens', 'target_tokens'])

### Visualize one sample

In [27]:
facts = sample['facts']
question = sample['question']
answer = tokenizer.decode(sample['target_tokens'])

background_text = tokenizer.batch_decode(sample['background_text'])

input_tokens = tokenizer.decode(sample['input_tokens'])

print(f"Facts: {' '.join(facts)}")
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"References: {' '.join(sample['references'])}")
print()
print('Background text: ', ' '.join(background_text))
print('Combined input: ', input_tokens)

print(f"Target: {answer}")


Facts: Mary moved to the bathroom. John went to the hallway.
Question: Where is Mary? 
Answer: bathroom
References: Mary moved to the bathroom.

Background text:   Troops are divided into five classes : Scouts, Shocktroopers, Engineers, Lancers and Armored Soldier. Troopers can switch classes by changing their assigned weapon. Changing class does not greatly affect the stats gained while in a previous class. With victory in battle, experience points are awarded to the squad, which are distributed into five different attributes shared by the entire squad, a feature differing from early games'method of distributing to different unit types.  = = Plot = =  The game takes place during the Second Europan War. Gallian Army Squad 422, also known as " The Nameless ", are a penal military unit composed of criminals, foreign deserters, and military offenders whose real names are erased from the records and thereon officially referred to by numbers. Ordered by the Gallian military to perform the m